# 1-step self-suppression

## Layout of beta

The parameter $beta$ is a $(1 + p \times N, N)$ matrix. Each column is a coefficient vector of one of $N$ neurons and
each coefficient vector consists of intercept and $p$ time-steps of coefficients of $N$ neurons.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%aimport model 
%aimport simulation

import os.path
from datetime import datetime
import numpy as np
from scipy import linalg
from pylab import *
from matplotlib.backends.backend_pdf import PdfPages
import h5py
from sklearn.decomposition.factor_analysis import FactorAnalysis

from model import *
import simulation

In [ ]:
np.random.seed(0)

T = 200
l = 1e-4
std = 1
p = 1
L = 2
N = 20

high = np.log(25 / T)
low = np.log(5 / T)

# simulate latent processes
# x, ticks = simulation.latents(L, T, std, l)
x = np.empty((T, L), dtype=float)
x[:T // 2, 0] = high
x[T // 2:, 0] = low
x[:, 1] = 2 * np.sin(np.linspace(0, 2 * np.pi * 5, T))
for l in range(L):
    x[:, l] -= np.mean(x[:, l])

# simulate spike trains
# a = np.empty((L, N), dtype=float)
a = 2 * np.random.rand(L, N) - 1
for l in range(L):
    a[l, :] /= linalg.norm(a[l, :]) / np.sqrt(N)

b = np.empty((1 + p * N, N))  # (1 + p)N * N matrix
b[0, :] = low
b[1:, :] = -10 * np.identity(N)

y, _, rate = simulation.spikes(x, a, b, intercept=True)
figure()
ylim(0, N)
for n in range(N):
    vlines(np.arange(T)[y[:, n] > 0], n, n + 1, color='black')
title('{} Spike trains'.format(N))
yticks(range(N))
gca().invert_yaxis()

In [ ]:
fa = FactorAnalysis(n_components=L)
m0 = fa.fit_transform(y)
a0 = fa.components_
# a0 = np.random.randn(L, N)
m0 *= np.linalg.norm(a0) / np.sqrt(N)
a0 /= np.linalg.norm(a0) / np.sqrt(N)

mu = np.zeros_like(x)

figure()
plot(m0)
title('Factor analysis')

In [ ]:
mu = np.zeros_like(x)
var = np.empty(L, dtype=float)
var[0] = 5
var[1] = 5
w = np.empty(L, dtype=float)
w[0] = 1e-3
w[1] = 1e-3

control = {'max iteration': 100,
           'fixed-point iteration': 3,
           'tol': 1e-5,
           'verbose': True}

lbound, m1, V1, a1, b1, new_var, new_scale, a0, b0, elapsed, converged = train(y, 1, mu, var, w,
                                                               a0=a0,
                                                               b0=None,
                                                               m0=m0,
                                                               V0=None,
                                                               guardV=False, guardSigma=False,
                                                               fixalpha=False, fixbeta=False, fixpostmean=False,
                                                               fixpostcov=False,
                                                               normofalpha=np.sqrt(N), intercept=True,
                                                               hyper=True,
                                                               control=control)

In [ ]:
c = linalg.lstsq(m1, x)[0]
rotated = np.dot(m1, c)
for l in range(L):
    figure()
    plot(x[:, l] - np.mean(x[:, l]), label='latent', color='blue')
    plot(rotated[:, l], label='transformed posterior', color='red')
    title('Rotated {}'.format(l + 1))